In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")

library(repr)
geneCutOff=4
targetCutOff=15
nFactors=15

In [ ]:
selCoefs <- read.csv("./../TextFiles/SignificantCoefMatrix.csv", row.names = 1)
selCoefs <- t(selCoefs)
KOGenes <-  colnames(selCoefs)
KOGenes <- sapply(KOGenes, function(x){strsplit(x,"_")[[1]][2]})
colnames(selCoefs) <- KOGenes

In [ ]:
guideModulesN <- data.frame(read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv"),  
                            row.names = 1)
rownames(guideModulesN) <- guideModulesN$GuideName 

head(guideModulesN)

In [ ]:
## This is for selecting training and validation sets in combo prediction
# allTestGuides = list()

# for(elem in unique(guideModulesN$GuideGroup)){
#     myTmp = guideModulesN[guideModulesN$GuideGroup == elem,]
#     k = sample(myTmp$GuideName, round(nrow(myTmp) *0.10, digits = 0), replace=FALSE)
#     allTestGuides = lappend(allTestGuides, k)
# }

# allTestGuides = unique(unlist(allTestGuides))
# allTrainingGuides = guideModulesN$GuideName[guideModulesN$GuideName %ni% allTestGuides]
# write.csv(allTestGuides, "./../TestGuides.csv")
# write.csv(allTrainingGuides, "./../TrainingGuides.csv")

In [ ]:
evaluateFactorNo <- function(){
    allTestGuides = list()

    for(elem in unique(guideModulesN$GuideGroup)){
        myTmp = guideModulesN[guideModulesN$GuideGroup == elem,]
        k = sample(myTmp$GuideName, round(nrow(myTmp) *0.2, digits = 0), replace=FALSE)
        allTestGuides = lappend(allTestGuides, k)
    }

    allTestGuides = unique(unlist(allTestGuides))
    allTrainingGuides = guideModulesN$GuideName[guideModulesN$GuideName %ni% allTestGuides]
    selCoefsTest = data.frame(selCoefs[,allTestGuides])
    selCoefsTrain = data.frame(selCoefs[,allTrainingGuides])
    
    
    allExpVars = list()

    for(i in 1:30){
        k <- icaimax(selCoefsTrain, nc = i, center = T)
        mySelFac = data.frame(k$S)
        
        r2ForNfac= c()
        for(j in 1:ncol(selCoefsTest)){

            mySelFac$y = selCoefsTest[,j]
            k = lm("y ~ .", data= mySelFac)
            myPredict = predict(k, newdata = mySelFac)

            myR2 = computeR2(mySelFac$y, myPredict)
            r2ForNfac = c(r2ForNfac, myR2)
        }
        
        allExpVars = lappend(allExpVars, r2ForNfac)
    }
    
    
    allExpVars = data.frame(do.call(cbind, allExpVars))
    rownames(allExpVars) = colnames(selCoefsTest) 
    colnames(allExpVars) = paste0("NFac_",1:ncol(allExpVars))
    allExpVars$genes = rownames(allExpVars)
    allExpVarsMelted = melt(allExpVars)
    
    return(allExpVarsMelted)

}

In [ ]:
hh <- evaluateFactorNo() 

for(i in 1:5){
    vv = evaluateFactorNo()
    
    hh = rbind(hh,  vv)
}


In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

pdf("./../SupplementaryFigures/S_5E.pdf", width=12, height=6)
ggplot(hh, aes(x=variable, y=value)) + 
     geom_violin(trim=FALSE)+
     geom_boxplot()+theme_minimal()+ 
     #stat_summary(fun.data = mean, geom = "text", size=6)+
        # stat_compare_means(method="t.test",paired = FALSE,
        #                         comparisons = list(c("Control", "KO")),
        #                         color="red", method.args = list(alternative = "less"),
        #                         label.y = c(30), aes(label = ..p.signif..))+
     #scale_fill_brewer(palette = "Pastel1")+
ylab("Explained variance in \n unseen perturbation responses (R^2)")+
theme(axis.text = element_text(size=15),
      axis.text.x = element_text(angle=90),
              axis.title =  element_text(size=16))+ylim(0,1) +xlab("Number of ICA factors") 

dev.off()